# Deep Nets with TF Abstractions

Let's explore a few of the various abstractions that TensorFlow offers. You can check out the tf.contrib documentation for more options.

# The Data

To compare these various abstractions we'll use a dataset easily available from the SciKit Learn library. The data is comprised of the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators. There are thirteen different
measurements taken for different constituents found in the three types of wine. We will use the various TF Abstractions to classify the wine to one of the 3 possible labels.

First let's show you how to get the data:

In [1]:
from sklearn.datasets import load_wine
wine_data = load_wine()
print(type(wine_data))

<class 'sklearn.utils.Bunch'>


The data is a sklearn.utils.Bunch object, which is very similar to a dictionary.

In [2]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

You can get a full description with **print(wine_data.DESCR)**. For now, let's go ahead and grab the features and the labels for the data.

In [3]:
feat_data = wine_data['data']
labels = wine_data['target']

### Train Test Split

As with any machine learning model, you should do some sort of test train split so you can evaluate your model's performance. Because this particular dataset is small, we'll just do a simple 70/30 train test split and we won't have any holdout data set.

Again, we'll use SciKit-Learn here for convienence:

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

### Scale the Data

With Neural Network models, its important to scale the data, again we can do this easily with SciKit Learn (I promise we'll get to TensorFlow soon!)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

Keep in mind we only fit the scaler to the training data, we don't want to assume we'll have knowledge of future test data. 

In [6]:
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)

# Abstractions

With our data set up, its now time to explore some TensorFlow abstractions! Let's start with the Estimator API, its one the abstractions featured in the official documentation tutorials.

## Estimator API

We first start by importing both tensorflow and the estimator API.

In [28]:
import tensorflow as tf
from tensorflow import estimator 

The estimator API can perform both Deep Neural Network Classification and Regression, as well as straight Linear Classification and Linear Regression. You can  

In [29]:
X_train.shape

(124, 13)

In [30]:
feat_cols = [tf.feature_column.numeric_column("x", shape=[13])]

In [39]:
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13],
                            feature_columns=feat_cols,
                            n_classes=3,
                            optimizer=tf.train.AdamOptimizer(learning_rate=0.001))




deep_model_2 = estimator.DNNClassifier(hidden_units=[13,13,13],
                            feature_columns=feat_cols,
                            n_classes=3,
                            optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.001))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpf07jhmtf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11495acf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpb5e493si', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_che

In [40]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},y=y_train,shuffle=True,batch_size=10,num_epochs=5)

In [33]:
deep_model.train(input_fn=input_fn,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpnx6bgqak/model.ckpt.
INFO:tensorflow:loss = 10.960678, step = 1
INFO:tensorflow:Saving checkpoints for 62 into /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpnx6bgqak/model.ckpt.
INFO:tensorflow:Loss for final step: 9.022005.


In [34]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},shuffle=False)

In [35]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpnx6bgqak/model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [36]:
predictions = [p['class_ids'][0] for p in preds]

In [37]:
from sklearn.metrics import confusion_matrix,classification_report

In [38]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        19
          1       0.49      0.95      0.65        22
          2       0.91      0.77      0.83        13

avg / total       0.42      0.57      0.46        54



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


____________
______________

In [41]:
deep_model_2.train(input_fn=input_fn,steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpb5e493si/model.ckpt.
INFO:tensorflow:loss = 9.298352, step = 1
INFO:tensorflow:Saving checkpoints for 62 into /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpb5e493si/model.ckpt.
INFO:tensorflow:Loss for final step: 10.447999.


In [42]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},shuffle=False)

In [43]:
preds = list(deep_model_2.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpb5e493si/model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [44]:
predictions = [p['class_ids'][0] for p in preds]

In [45]:
from sklearn.metrics import confusion_matrix,classification_report

In [46]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        19
          1       0.42      1.00      0.59        22
          2       1.00      0.08      0.14        13

avg / total       0.41      0.43      0.27        54



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
